In [1]:
# DS
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

import gensim
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

from my_utils import *

In [2]:
pd.options.mode.chained_assignment = None  # default='warn' # disable chain assignment warnings

p_data = "data"

print(VERSION)

v3.0


# POI2Vec

In [3]:
gdf_zoningm = read_csv_to_gdf(f"result/gdf_zoningm_{VERSION}.csv")
gdf_poi = read_csv_to_gdf(f"result/gdf_poi_{VERSION}.csv")

##  generate sequences

### Option 1. Distance Greedy 

In [ ]:
sequences = get_sequences_by_distancegreedy(gdf_zoningm, gdf_poi)

  0%|                                                                                         | 0/2732 [00:00<?, ?it/s]

In [ ]:
np.save(f"result/Sequences_greedy_{VERSION}.npy", sequences)

In [ ]:
len(sequences.keys()),len(gdf_zoningm)

##